In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [3]:
def create_column_index(df):
    #takes dataframe, and returns a dictionary of replacing each column top 'unnamed' in the first row as it's corresponding category
    #note that column indexes will have two rows, data category and specific statistic
    columns = [x for x in df.columns]
    current = columns[0]
    for i in range(len(columns)):
        if columns[i][:7] == 'Unnamed':
            columns[i] = current
        else:
            current = columns[i]
    column_dict = {x : y for x,y in zip(df.columns,columns)}
    return column_dict

In [4]:
def multi_index(df):
    #creates the multi-index for a given dataframe
    df.rename(columns=create_column_index(df),inplace=True)
    column_names = pd.MultiIndex.from_tuples(zip(df.columns,df.iloc[0,:]))
    df.columns = column_names
    return df

In [5]:
def create_master_df(file_loc,initialize_file):
    #takes file name and location and initializes the empty master dataframe
    master_df = pd.read_excel('./{}/{}'.format(file_loc,initialize_file),'Ranked Measure Data')
    master_df = master_df.iloc[0:1]
    master_df.rename(columns={'Unnamed: 0':'FIPS', 'Unnamed: 1': 'State', 'Unnamed: 2' : 'County'}, inplace=True)
    master_df.insert(0,"Year",['Year'], allow_duplicates=True)
    master_df = multi_index(master_df)
    master_df.drop(index=range(0, len(master_df)), inplace=True)
    return master_df

In [6]:
def get_frames(file_loc):
    #pulls data out from folder, and labels them as their year
    #output is a dictionary with each dataframe, with key as the string of the year that it's from
    output = {}
    for file in os.listdir(file_loc):
        year = file[:4]
        output[year] = pd.read_excel('./{}/{}'.format(file_loc,file),'Ranked Measure Data')
        output[year].rename(columns={'Unnamed: 0':'FIPS', 'Unnamed: 1': 'State', 'Unnamed: 2' : 'County'}, inplace=True)
        
        year_data = ["Year"]+[year for i in range(output[year].shape[0]-1)]
        output[year].insert(0,'Year', year_data, allow_duplicates=True)

    return output

In [7]:
def get_all_columns(data):
    #helper for column_intersection method below
    #returns column names when given a dict full of dataframes
    assert type(data) == dict, 'input must be dictionary'
    output_dict = {}
    for year,frame in data.items():
        columns = [x for x in frame.columns]
        output_dict[year] = columns
    return output_dict

In [8]:
def column_intersection(data):
    #called by the populate_master_df method below
    #returns the columns that exist across all of the dataframes
    assert type(data) == dict, 'input must be dictionary'
    column_dictionary = get_all_columns(data)
    output_set = set()
    for year, column_names in column_dictionary.items():
        if len(output_set) == 0:
            output_set = set(column_names)
        else:
            output_set = output_set.intersection(column_names)
    return output_set

In [9]:
def populate_master_df(master_df, file_loc):
    #builds the full dataframe

    data = get_frames(file_loc)
    #pull down the data from each sheet into a dictionary with keys as the string form of the years
    
    for year, frame in data.items():
        data[year] = multi_index(frame)
    #multi-index each frame

    kept_columns = list(column_intersection(data))
    #column intersection, data is a set type 

    master_df = master_df[kept_columns]

    for year, frame in data.items():
        #iterate through each frame that we have

        data[year] = data[year][kept_columns]
        data[year] = data[year].reindex(columns = kept_columns)
        #remove columns that don't exist across all frames

        master_df = pd.concat([master_df,data[year]],ignore_index=True) #, on=master_df.columns
        #consolidate

    return master_df

        

In [10]:
data_folder = 'LBW and Other Data'
init_file_name = '2022 County Health Rankings Data - v1.xlsx'
master_df = create_master_df(data_folder,init_file_name)
master_df = populate_master_df(master_df, data_folder)

In [11]:
reorder = [ (                     'Year',                  'Year'),
            (                   'County',                'County'),
            (                    'State',                 'State'),
            ('Poor physical health days',         '95% CI - High'),
		    ('Poor physical health days',          '95% CI - Low'),
            (      'Poor or fair health',         '95% CI - High'),
            (      'Poor or fair health',          '95% CI - Low'),
            (  'Poor mental health days',         '95% CI - High'),
            (  'Poor mental health days',          '95% CI - Low'),
            (            'Adult smoking',         '95% CI - High'),
            (            'Adult smoking',          '95% CI - Low'),
            (            'Adult smoking',             '% Smokers'),
            (            'Adult obesity',         '95% CI - High'),
            (            'Adult obesity',          '95% CI - Low'),
            (          'Low birthweight',         '95% CI - High'),
            (          'Low birthweight',          '95% CI - Low'),
            (          'Low birthweight',            'Unreliable'),
            (      'Children in poverty',         '95% CI - High'),
            (      'Children in poverty',          '95% CI - Low'),
            (      'Children in poverty', '% Children in Poverty'),

            ]

In [12]:
master_df = master_df.reindex(columns=reorder)

In [13]:
master_df = master_df.loc[master_df[('Year','Year')] != 'Year',:]
#removing excess labels

In [14]:
col_names = list(master_df.columns)
new_col_names = []
for i in col_names:
    new_col_names.append((str(i[0]).lower(),str(i[1]).lower()))
master_df.columns = pd.MultiIndex.from_tuples(new_col_names)

In [15]:
cali_data = master_df[master_df[('state','state')] == 'California']

In [16]:
cali_data.reset_index(drop=True,inplace=True)

In [17]:
cali_data[('low birthweight','unreliable')].fillna(0,inplace=True)
cali_data[('low birthweight','unreliable')].replace('x',1,inplace=True)
#useful data that will be used to eventually impute the mean across the state

In [18]:
for col in cali_data.columns[3:]:
    cali_data[col] = cali_data[col].astype('float32')


C:\Users\Jack Koltermann\AppData\Local\Temp\ipykernel_26516\3910101897.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cali_data[col] = cali_data[col].astype('float32')


In [19]:
cali_data.dtypes

year                       year                      object
county                     county                    object
state                      state                     object
poor physical health days  95% ci - high            float32
                           95% ci - low             float32
poor or fair health        95% ci - high            float32
                           95% ci - low             float32
poor mental health days    95% ci - high            float32
                           95% ci - low             float32
adult smoking              95% ci - high            float32
                           95% ci - low             float32
                           % smokers                float32
adult obesity              95% ci - high            float32
                           95% ci - low             float32
low birthweight            95% ci - high            float32
                           95% ci - low             float32
                           unreliable   

In [20]:
for col in list(cali_data.columns[3:]):
    cali_data[col].fillna(cali_data[col].mean(),inplace=True)

In [21]:
cali_data['year'] = cali_data['year'].astype('float32')

C:\Users\Jack Koltermann\AppData\Local\Temp\ipykernel_26516\3027082188.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cali_data['year'] = cali_data['year'].astype('float32')


In [22]:
def verify_ci(col_name):
    if col_name[1][-4:] == 'high' or col_name[1][-3:] == 'low':
        return True
    return False

In [23]:
def organize_column_ci(df):
    organized_ci = {}
    for col in df.columns:
        if verify_ci(col):
            parent_category = col[0]
            child_category = col[1]
            if not parent_category in organized_ci.keys():
                organized_ci[parent_category] = []
                organized_ci[parent_category].append(df[col].values)
            else:
                organized_ci[parent_category].append(df[col].values)
        else:
            continue
    
    return organized_ci

In [24]:
def calc_parent_mean_series(df):
    ci_data = organize_column_ci(df)
    output = {}
    for key, series in ci_data.items():
        series_a = series[0]
        series_b = series[1]
        output[key] = [(x+y)/2 for x,y in zip(series_a,series_b)]
    return output

In [25]:
def add_mean_series(df):
    mean_series = calc_parent_mean_series(df)
    for key,value in mean_series.items():
        df[(key,str(key)+' series_mean')] = value
    return df

In [26]:
cali_data = add_mean_series(cali_data)

C:\Users\Jack Koltermann\AppData\Local\Temp\ipykernel_26516\2328983087.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[(key,str(key)+' series_mean')] = value
C:\Users\Jack Koltermann\AppData\Local\Temp\ipykernel_26516\2328983087.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[(key,str(key)+' series_mean')] = value
C:\Users\Jack Koltermann\AppData\Local\Temp\ipykernel_26516\2328983087.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

In [27]:
cali_data.head()

year     county       state poor physical health days               \
     year     county       state             95% ci - high 95% ci - low   
0  2010.0    Alameda  California                  3.490000     2.710000   
1  2010.0     Alpine  California                  4.431681     3.361643   
2  2010.0     Amador  California                  5.520000     2.030000   
3  2010.0      Butte  California                  4.930000     3.130000   
4  2010.0  Calaveras  California                  6.720000     2.190000   

  poor or fair health              poor mental health days               \
        95% ci - high 95% ci - low           95% ci - high 95% ci - low   
0           15.650000    11.620000                3.840000      3.08000   
1           20.095619    15.234496                4.466881      3.34523   
2           27.920000    10.450000                5.310000      1.69000   
3           18.170000    10.420000                4.470000      2.77000   
4           24.680000     7.230000                3.690000      0.86000   

  adult smoking  ... children in poverty                                     \
  95% ci - high  ...       95% ci - high 95% ci - low % children in poverty   
0     15.620000  ...           15.400000    12.600000             14.000000   
1     16.279007  ...           28.799999    18.400000             23.600000   
2     16.279007  ...           16.299999    10.700000             13.500000   
3     24.110001  ...           24.200001    18.200001             21.200001   
4     16.279007  ...           17.400000    11.400000             14.400000   

              poor physical health days             poor or fair health  \
  poor physical health days series_mean poor or fair health series_mean   
0                              3.100000                       13.635000   
1                              3.896662                       17.665058   
2                              3.775000                       19.184999   
3                              4.030000                       14.295000   
4                              4.455000                       15.955000   

              poor mental health days             adult smoking  \
  poor mental health days series_mean adult smoking series_mean   
0                            3.460000                 13.565001   
1                            3.906055                 13.998363   
2                            3.500000                 13.998363   
3                            3.620000                 19.275000   
4                            2.275000                 13.998363   

              adult obesity             low birthweight  \
  adult obesity series_mean low birthweight series_mean   
0                 19.400000                    7.020000   
1                 24.849998                    6.384604   
2                 27.150000                    4.575000   
3                 25.500000                    5.695000   
4                 23.400002                    5.995000   

              children in poverty  
  children in poverty series_mean  
0                       14.000000  
1                       23.599998  
2                       13.500000  
3                       21.200001  
4                       14.400000  

[5 rows x 27 columns]

In [87]:
def create_time_series(df):
    time_series = {}
    for col in df.iloc[:,3:].columns:
        if col[1][-11:] == 'series_mean':
            time_series[col] = {}
            for county in df[('county','county')].unique():
                time_series[col][county] = {}
                for year in df[('year','year')].unique():
                    time_series[col][county][year] = df[(df[('county','county')] == county) & (df[('year','year')] == year)][col]
    return time_series

In [88]:
t = create_time_series(cali_data)

In [91]:
t[('children in poverty','children in poverty series_mean')]

{'Alameda': {2010.0: 0    14.0
  Name: (children in poverty, children in poverty series_mean), dtype: float64,
  2011.0: 58    12.9
  Name: (children in poverty, children in poverty series_mean), dtype: float64,
  2012.0: 116    17.200001
  Name: (children in poverty, children in poverty series_mean), dtype: float64,
  2013.0: 175    16.799999
  Name: (children in poverty, children in poverty series_mean), dtype: float64,
  2014.0: 233    17.299999
  Name: (children in poverty, children in poverty series_mean), dtype: float64,
  2015.0: 291    15.400001
  Name: (children in poverty, children in poverty series_mean), dtype: float64,
  2016.0: 349    14.3
  Name: (children in poverty, children in poverty series_mean), dtype: float64,
  2017.0: 407    13.8
  Name: (children in poverty, children in poverty series_mean), dtype: float64,
  2018.0: 465    12.4
  Name: (children in poverty, children in poverty series_mean), dtype: float64,
  2019.0: 523    10.5
  Name: (children in poverty, ch